In [6]:
import pandas as pd     # Data frame minipulation
import psycopg2 as pg   # Postgres db connection
import re
import matplotlib.pyplot as plt

from contextlib import redirect_stdout # For capture the exec output
import io # Store into a string stream
import os
import requests
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

## DB fetching

In [8]:
# DB Connection stuff
DB = os.getenv("DB_NAME")
USER = os.getenv("DB_USER")
PASS =  os.getenv("DB_PASSWORD")

connection = pg.connect(database=DB, user=USER, password=PASS, host="localhost")

In [9]:
sql_str = "SELECT * FROM public.aggregations;"
df = pd.read_sql(sql=sql_str, con=connection)

C:\Users\DELL\AppData\Local\Temp\ipykernel_4960\1554223786.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sql_str, con=connection)


In [10]:
df

,id,shift_date,shift_name,machine_name,production,rejection,machine_runtime_minute,machine_downtime_minute,ideal_speed,actual_speed,availability,performance,quality,oee
0,1,2024-06-02,I,Machine 1,8656,0,345.0,510.0,52.0,50.367954,68.0,48.0,100.0,32.0
1,2,2024-06-02,I,Machine 2,8000,0,188.0,510.0,45.0,41.443200,37.0,95.0,100.0,35.0
2,3,2024-06-02,I,Machine 3,18200,15,362.0,510.0,55.0,46.398834,71.0,91.0,100.0,64.0
3,4,2024-06-02,I,Machine 4,8137,10,377.0,510.0,45.0,42.208706,74.0,48.0,100.0,35.0
4,5,2024-06-02,I,Machine 5,8430,0,255.0,510.0,70.0,64.895020,50.0,47.0,100.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,86,2024-06-10,III,Machine 1,16128,0,312.0,420.0,52.0,50.954200,74.0,99.0,100.0,73.0
86,87,2024-06-10,III,Machine 2,16297,11,302.0,420.0,45.0,41.986385,72.0,120.0,100.0,86.0
87,88,2024-06-10,III,Machine 3,12351,30,196.0,420.0,55.0,46.075150,47.0,115.0,100.0,54.0
88,89,2024-06-10,III,Machine 4,8507,13,191.0,420.0,45.0,40.682995,45.0,99.0,100.0,44.0


## Random Data trail

In [2]:
import pandas as pd
import numpy as np

# Create random data for the DataFrame
np.random.seed(42)
emp_id = np.arange(1001, 1011)  # Employee IDs from 1001 to 1010
emp_name = ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'Charlie Black', 
            'Daisy Green', 'Edward White', 'Fiona Blue', 'George Pink', 'Helen Yellow']
location = np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'], size=10)
salary = np.random.randint(50000, 120000, size=10)

# Create DataFrame
df = pd.DataFrame({
    'emp_id': emp_id,
    'emp_name': emp_name,
    'location': location,
    'salary': salary
})

df

,emp_id,emp_name,location,salary
0,1001,John Doe,Houston,94131
1,1002,Jane Smith,Phoenix,110263
2,1003,Alice Johnson,Chicago,66023
3,1004,Bob Brown,Phoenix,91090
4,1005,Charlie Black,Phoenix,117221
5,1006,Daisy Green,Los Angeles,114820
6,1007,Edward White,Chicago,50769
7,1008,Fiona Blue,Chicago,109735
8,1009,George Pink,Chicago,112955
9,1010,Helen Yellow,Phoenix,114925


## Final workflow

In [5]:
sys_prompt = f"""
You are a data analyst, and you're job is to write python code to evaluate the users query
you have access to the following tools, please note you can ONLY write python code:
 - pandas imported as pd
 - matplotlib imported as plt
 - a data frame df with columns ({df.columns})
 - if any plots are made they must be saved as png files in plots directory
 
YOU MAY NOT INSTALL ANY ADDITIONAL PACKAGES AND IMPORT ANY PACKAGES
"""

message_history = [
	{"role": "system", "content": sys_prompt},
	{"role": "user", "content": "Make a plot of average production by machine"}
]

### Hugging Face Inference API (~5s per query)

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(token=os.getenv("HF_SECRET"))

out = client.chat_completion(
	model="microsoft/Phi-3.5-mini-instruct",
	messages=message_history,
	max_tokens=500,
	stream=False,
)

raw_out = out.choices[0].message.content

print(raw_out)

ModuleNotFoundError: No module named 'huggingface_hub'

### Ollama API (local model: ~50s per query)

In [8]:
from openai import OpenAI # Running model on OLLAMA requires ollama to be installed 


client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

chat_completion = client.chat.completions.create(
    messages=message_history,
    model='vaibhav-s-dabhade/phi-3.5-mini-instruct', # Model needs to be downloaded first
    max_tokens=600
)

raw_out = chat_completion.choices[0].message.content
print(raw_out)

To create a bar chart that shows the average production for each machine using pandas and matplotlib, you can follow these steps in Python code:

```python
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame with all necessary data columns correctly populated 

# Calculate average production by machine
average_production = df.groupby('machine_name')['production'].mean().reset_index()

# Plotting the bar chart for each machine using matplotlib
plt.figure(figsize=(10, 6)) # Set figure size if needed
plt.bar(range(len(average_production)), average_production['production'], color='blue')

# Adding labels and title to your plot - Ensure 'plots' directory exists or create it using os module before saving the file
plt.xlabel('Machine Name Index', fontsize=14)  # X-axis label with specified size for readability
plt.ylabel('Average Production (units)', fontsize=14)    # Y-axis label indicating what we plot on it, similarly formatted as above
plt.title('Average

## Execute the genrated code

In [9]:
pattern = r"```python\n([\s\S]*?)\n```"
matches = re.search(pattern, raw_out)
code: str = matches.groups()[0]

# ss = io.StringIO()
# with redirect_stdout(ss):
# exec(code)

# output = ss.getvalue()

# print(f"LLM Code output was:\n {output}")
# for file in listdir("plots"):
#     print(f"LLM saved plot {file}")
exec(code)